In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [8]:
spark = SparkSession.builder.appName('TransformationStateSpark').getOrCreate()

In [9]:
df_state = spark.read.csv('storage/PRECOS SEMESTRAIS - AUTOMOTIVOS.csv', header=True, inferSchema=True, sep=';')

distinct_state = df_state.select('Estado - Sigla').distinct().collect()

for row in distinct_state:
    state = row['Estado - Sigla']
    df_state_filter = df_state.filter(col('Estado - Sigla') == state)
    df_state_filter = df_state_filter.withColumn('Data da Carga ETL', current_date())
    df_state_filter = df_state_filter.withColumn('Valor de Venda', regexp_replace(col('Valor de Venda'), ',', '.'))
    
    #Exibe registros que foram segregados por estado 
    df_state_filter.show()
    #Salva registros segregandos por estado
    df_state_filter.write.csv(f'storage/state/{state}/full/', header=True, mode="overwrite")

    #Agregando registros por estado e demais
    df_state_agg = df_state_filter.groupBy(['Estado - Sigla', 'Data da Coleta', 'Produto', 'Bandeira']).agg(round(avg('Valor de Venda'), 2).alias('Valor de Venda'))
    #Exibe registros que foram agregados
    df_state_agg.show()
    #Salva registros agregados por estado
    df_state_agg.write.csv(f'storage/state/{state}/agg/', header=True, mode="overwrite")

spark.stop()

+--------------+--------------+---------+--------------------+-------------------+--------------------+----------+-----------+----------------+---------+------------------+--------------+--------------+---------------+-----------------+-------------+-----------------+
|Regiao - Sigla|Estado - Sigla|Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|Complemento|          Bairro|      Cep|           Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|     Bandeira|Data da Carga ETL|
+--------------+--------------+---------+--------------------+-------------------+--------------------+----------+-----------+----------------+---------+------------------+--------------+--------------+---------------+-----------------+-------------+-----------------+
|             S|            SC| BLUMENAU|  AUTO POSTO NB LTDA| 03.067.669/0001-99|RUA FREI ESTANISL...|       676|          0|           ASILO|89037-001|        DIESEL S10|    05/01/2023|      